# Novozymes Difference + Surface Area Ensemble

Ensemble using two concepts based on directly predicting tm from features of the individual mutatant sequences in the test data (i.e. training data is not used).

Difference features comes from the insightful notebook here: https://www.kaggle.com/code/cdeotte/difference-features-lb-0-600/notebook

Acessible surface area computation with blosum62 substitutions: https://www.kaggle.com/code/tilii7/surface-area-scaled-by-blosum62-substitutions/notebook

Based on 5 single point tests, best ensemble of these two models cosists of using the blend ratio that matches the individual scores, normalized sum to 1.
* Individual scores: .415, .397
* Ensemble score: .463

Next steps, add a third model to ensemble.  
* Train based model, either with:
    * Kaggle data
    * External data
* Other models based on test features

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

import Bio
from Bio.PDB import PDBParser
from Bio.PDB.SASA import ShrakeRupley  # what is this?
from scipy.stats import rankdata


In [ ]:
test = pd.read_csv('../input/novozymes-enzyme-stability-prediction/test.csv')
deletions = test.loc[test.protein_sequence.str.len()==220,'seq_id'].values
test.head()

In [ ]:
# LOAD TEST WILDTYPE
base = 'VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK'
len(base)

In [ ]:
def get_test_mutation(row):
    for i,(a,b) in enumerate(zip(row.protein_sequence,base)):
        if a!=b: break
    row['wildtype'] = base[i]
    row['mutation'] = row.protein_sequence[i]
    row['position'] = i+1
    return row
test = test.apply(get_test_mutation,axis=1)
test.loc[test.seq_id.isin(deletions),'mutation'] = '_'
test.head()

In [ ]:
!pip install biopandas
from biopandas.pdb import PandasPdb

In [ ]:
# TEST WILD TYPE
atom_df0 = PandasPdb().read_pdb(f'../input/nesp-kvigly-test-mutation-pdbs/WT_unrelaxed_rank_1_model_3.pdb')
atom_df0 = atom_df0.df['ATOM']
wt = atom_df0.groupby('residue_number').b_factor.agg('first').values

Plot difference of plDDT between wild type and row 1 mutation

In [ ]:
for index,row in test.iterrows():
    aa1 = row.wildtype; aa2 = row.mutation; pos = row.position
    atom_df = PandasPdb().read_pdb(f'../input/nesp-kvigly-test-mutation-pdbs/{aa1}{pos}{aa2}_unrelaxed_rank_1_model_3.pdb')
    atom_df = atom_df.df['ATOM']
    mut = atom_df.groupby('residue_number').b_factor.agg('first').values
    break

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(range(221),wt,label='wild type')
plt.plot(range(221),mut,label='mutant')
plt.legend()
plt.title('Test Wildtype Row 1 vs. Test Mutant Row 1',size=18)
plt.show()

In [ ]:
diffs = []
missing = []

for index,row in test.iterrows():
    #print(index,', ',end='')
    aa1 = row.wildtype
    aa2 = row.mutation
    pos = row.position
    d = 0
    try:
        atom_df = PandasPdb().read_pdb(f'../input/nesp-kvigly-test-mutation-pdbs/{aa1}{pos}{aa2}_unrelaxed_rank_1_model_3.pdb')
        atom_df = atom_df.df['ATOM']
        mut = atom_df.groupby('residue_number').b_factor.agg('first').values
        d = mut[pos-1] - wt[pos-1]
    except:
        missing.append(index)
        
    diffs.append(d)
print('These PDB are missing:', missing )

In [ ]:
sns.histplot(diffs)

## Surface Area

In [ ]:
test_wt_pdb_path = "../input/novozymes-enzyme-stability-prediction/wildtype_structure_prediction_af2.pdb"
test = pd.read_csv("../input/novozymes-enzyme-stability-prediction/test.csv")
test_w_extra_info = pd.read_csv("../input/new-novoesp-train-and-test-data/new_test.csv")
ss = pd.read_csv("../input/novozymes-enzyme-stability-prediction/sample_submission.csv")
test = test.merge(test_w_extra_info[["seq_id", "edit_type", "edit_idx", "wildtype_aa", "mutant_aa"]], on="seq_id", how="left").fillna(-1)

# add blosum from: https://www.kaggle.com/code/tilii7/surface-area-scaled-by-blosum62-substitutions/notebook
blosum = pd.read_csv('../input/blosum62-scaled/BLOSUM62_scaled.csv')
blosum.set_index('amino-acids', inplace=True)
# Wildtype test sequence 
wildtype_aa = "VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK"


test

In [ ]:
pdb_parser = PDBParser(QUIET=1)

def get_sr_sasa(pdb_path, return_struct=False, pdb_identifier="AF70", sr_n_points=250):
    sr = ShrakeRupley(n_points=sr_n_points)
    struct = pdb_parser.get_structure(pdb_identifier, pdb_path)
    sr.compute(struct, level="R")
    if return_struct:
        return struct
    else:
        return [x.sasa for x in struct.get_residues()]
    
sasa_by_residue = get_sr_sasa(test_wt_pdb_path)
residue_idx_to_sasa = {i:_sasa for i, _sasa in enumerate(sasa_by_residue)}
test["sasa"] = test["edit_idx"].apply(lambda x: residue_idx_to_sasa.get(x, 0.0))
test

In [ ]:
# Multiply SASA values by substitution scores
for k in range(test.shape[0]):
    if test['edit_type'][k] == 'substitution':
        test['sasa'][k] = test['sasa'][k] * blosum[test['wildtype_aa'][k]][test['mutant_aa'][k]]

# Set deletions to the lowest score of all mutations at a given position
for k in range(test.shape[0]):
    if test['edit_type'][k] == 'deletion':
        test['sasa'][k] = test[test['edit_idx'] == test['edit_idx'][k]]['sasa'].values.min()

print(test.head())

## Submission

In [ ]:
# Single diffs submission
ss = pd.read_csv('../input/novozymes-enzyme-stability-prediction/sample_submission.csv')
ss['tm'] = diffs
#ss.to_csv('diffs_submission.csv',index=False)
sns.histplot(ss.tm)


In [ ]:
print(f'max,min of diffs: {min(diffs)},{max(diffs)}')

In [ ]:
ranked_diffs = rankdata(diffs)
sns.histplot(ranked_diffs)

In [ ]:
ranked_diffs

In [ ]:
# Single sasa submission
# This will score 0.397, and .415 with blossom62
sasa = test['sasa']
ss["tm"] = test["sasa"]
#ss.to_csv("sasa_submission.csv", index=False)
sns.histplot(ss.tm)

In [ ]:
print(f'max,min of sasa: {min(sasa)},{max(sasa)}')

In [ ]:
ranked_sasa = rankdata(sasa)
sns.histplot(ranked_sasa,bins=50)

In [ ]:
# Ensemble solution

ss.tm = (.415 * ranked_diffs + .584 * ranked_sasa)/len(ss)
ss.to_csv('submission.csv',index=False)

sns.histplot(ss.tm)
# best = pd.read_csv('../input/nesp-thermonet-v2/ensemble_submission.csv')
# best.tm = rankdata( best.tm )
# best.head()

# submission = sub.copy()
# submission.tm = (0.15 * rankdata(sub.tm) + 0.85 * rankdata(best.tm))/len(submission)
# plt.hist( submission.tm, bins=100)
# plt.show()
# submission.to_csv('submission_ensemble.csv',index=False)